고정실,김가람,김득실,김영현,김충연,민병춘1,박주연1,박주연2,벌,안중훈,윤병시,이미우,임석봉,전창희,정광훈2,정금례,정용태,이귀임,정복연,김정한,정광훈1,조진욱

In [ ]:
import mne
import numpy as np
import pandas as pd
import scipy.stats as stats
import openpyxl as op

# Define Cohens'D
def cohens_d(group1, group2):
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    n1, n2 = len(group1), len(group2)
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    return (mean1 - mean2) / pooled_std

# 테스트 실행 함수
def run_tests(before_mean, after_mean, band_name, wb, row, name):
    print(f"\n=== {band_name} 대역 분석 결과 ===")

    # mean of before vs after
    mean_before = np.mean(before_mean)
    mean_after = np.mean(after_mean)
    print(f"mean_before: {mean_before}")
    print(f"mean_after: {mean_after}")

    # Kolmogorov-Smirnov test
    difference = before_mean - after_mean
    ks_statistic, ks_p_value = stats.kstest(difference, stats.norm.cdf)
    print(f"KS Statistic: {ks_statistic}")
    print(f"KS p-value: {ks_p_value}")

    # Paired t-test
    t_stat, p_value = stats.ttest_rel(before_mean, after_mean)
    print(f"t_statistics: {t_stat}")
    print(f"p-value: {p_value}")

    # Calculate Effect Size
    effect_size = cohens_d(before_mean, after_mean)
    print(f"Effect size: {effect_size}")

    # Excel 저장
    ws_b = wb["Sheet2"]
    band_row_offset = {
        'Delta': -3,
        'Theta': -2,
        'Alpha': -1,
        'Beta': 0,
        'Gamma': 1
    }
    
    row_b = row * 5 + band_row_offset.get(band_name)
    if row_b is None:
        print(f"Invalid band name: {band_name}")
        return

    ws_b.cell(row=row_b, column=1).value = name
    ws_b.cell(row=row_b, column=4).value = mean_before
    ws_b.cell(row=row_b, column=5).value = mean_after
    ws_b.cell(row=row_b, column=6).value = ks_statistic
    ws_b.cell(row=row_b, column=7).value = ks_p_value
    ws_b.cell(row=row_b, column=9).value = t_stat
    ws_b.cell(row=row_b, column=10).value = p_value
    ws_b.cell(row=row_b, column=12).value = effect_size

# Removing large artifact by band 함수 정의
def remove_artifacts(before_mean, after_mean, large_artifact):
    before_idx = set()
    after_idx = set()

    for t_start, t_end in large_artifact:
        if t_start < 3600 and t_end < 3600:
            before_idx.update(range(t_start*200, t_end*200))
            after_idx.update(range(t_start*200, t_end*200))
        elif t_start < 3600 and t_end >= 3600:
            before_idx.update(range(t_start*200, 3600*200))
            after_idx.update(range(t_start*200, 3600*200))
            after_idx.update(range(0, (t_end-3600)*200))
            before_idx.update(range(0, (t_end-3600)*200))
        else:
            after_idx.update(range((t_start-3600)*200, (t_end-3600)*200))
            before_idx.update(range((t_start-3600)*200, (t_end-3600)*200))

    before_idx = sorted(before_idx)
    after_idx = sorted(after_idx)

    return np.delete(before_mean, before_idx), np.delete(after_mean, after_idx)

# Process band data 함수 정의
def process_band_data(tfr_data, large_artifact):
    start1, end1 = 0, 3599 * 200
    start2, end2 = 3600 * 200, 7199 * 200

    # Split before and after
    before = tfr_data[:, :, start1:end1]
    after = tfr_data[:, :, start2:end2]

    # Average over channels and frequencies
    before_mean_channel = before.mean(axis=0)
    after_mean_channel = after.mean(axis=0)
    
    before_mean = before_mean_channel.mean(axis=0)
    after_mean = after_mean_channel.mean(axis=0)

    # Remove artifacts
    return remove_artifacts(before_mean, after_mean, large_artifact)

# Main 함수 정의
def main():
    # Initialize name dictionary
    names = ['고정실', '김가람', '김득실', '김영현', '김충연', '민병춘1', '박주연1', '박주연2', '벌', 
             '안중훈', '윤병시', '이미우', '임석봉', '전창희', '정광훈2', '정금례', '정용태', '이귀임', 
             '정복연', '김정한', '정광훈1', '조진욱']
    name_dict = {name: i+1 for i, name in enumerate(names)}

    # Get analysis targets
    name_list = input("분석대상 이름을 입력하세요. (여러명일 경우 쉼표로 구분)").split(',')
    print(f'분석대상은 {name_list}입니다.')

    # Load workbook and artifact data
    wb = op.load_workbook(r"C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\Mg_infusion_data.xlsx")
    large_artifact_data = pd.read_csv(r'C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\large_artifact.csv', encoding='utf-8-sig')

    # Process each name
    for name in name_list:
        print(f'{name}의 데이터를 분석합니다.')
        
        # Get large artifacts
        large_artifact = []
        if name in large_artifact_data['Name'].values:
            coordinates = large_artifact_data[large_artifact_data['Name'] == name]['Coordinates'].values[0]
            if coordinates != 'skip':
                large_artifact = eval(coordinates)
        print(f"{name}의 large artifact: {large_artifact}")

        # Get row number
        row = name_dict.get(name)
        if row is None:
            print(f"{name}이(가) name_dict에 없습니다.")
            exit()

        # Store band data in dictionary
        tfr_data = {}
        
        # Load band data
        for band in ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma1', 'Gamma2']:
            file_path = rf'H:\Mg_EEG\tfr_files_gamma\{name}{band}_7200_tfr.h5'
            tfr = mne.time_frequency.read_tfrs(file_path)
            tfr_data[band] = tfr.data
            del tfr
        print(f"{name}의 데이터를 불러왔습니다.")

        # Combine Gamma bands
        if 'Gamma1' in tfr_data and 'Gamma2' in tfr_data:
            tfr_data['Gamma'] = np.concatenate([tfr_data['Gamma1'], tfr_data['Gamma2']], axis=1)
            del tfr_data['Gamma1'], tfr_data['Gamma2']
        print(f"{name}의 Gamma1과 Gamma2 데이터를 결합했습니다.")

        # Process each band
        for band in ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']:
            if band not in tfr_data:
                print(f"Warning: {band} data not found")
                exit()
                
            before_clean, after_clean = process_band_data(tfr_data[band], large_artifact)
            print(f"{band} 데이터의 artifact를 제거했습니다.")
            
            run_tests(before_clean, after_clean, band, wb, row, name)
            print(f"{band} 데이터의 테스트를 실행했습니다.")
            
            del tfr_data[band]
            print(f"{band} 데이터를 삭제했습니다.")

    # Save results
    wb.save(r"C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\Mg_infusion_data.xlsx")
    print(f"{name_list}의 데이터 분석결과를 저장했습니다.")
    print(f"{name_list}의 데이터 분석이 완료되었습니다.")

if __name__ == "__main__":
    main()